In [1]:
import os
import cv2
import numpy as np
from cvl.dataset import OnlineTrackingBenchmark
from cvl.trackers import MOSSETracker, DCFMOSSETracker
from cvl.lib import get_roi, resume_performance, get_arguments
import matplotlib.pyplot as plt
from copy import copy, deepcopy
import torch
from tqdm.notebook import tqdm as tqdm
import json

In [2]:
ds_path = "Mini-OTB/"

In [3]:
seq_list = os.listdir(ds_path)

In [4]:
dataset = OnlineTrackingBenchmark(ds_path)

# 1) Choose the tracker

In [5]:
TRACKER = "mobilenet" # ["mosse", "resnet", "mobilenet", "alexnet", "vgg16", "hand_f"]

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [7]:
if TRACKER == "mosse":
    tracker = MOSSETracker()
    squared = False
elif TRACKER in ["resnet", "mobilenet", "alexnet", "vgg16", "hand_f"]:
    tracker = DCFMOSSETracker(dev = device, features = TRACKER)
    squared = True

In [8]:
results = {}

In [9]:
for idx, seq in enumerate(dataset):
    print(f"----------------------------Evaluating video named {seq.sequence_name}----------------------------------")
    bboxes = []
    for frame_idx, frame in enumerate(seq):
        image_color = frame['image']
        if TRACKER == "mosse":
            image = np.sum(image_color, 2) / 3 # grayscale
        else:
            image = np.transpose(np.float64(image_color), (2, 0, 1))
        if frame_idx == 0:
            bbox = copy(frame['bounding_box'])
            aaaa = bbox
            if bbox.width % 2 == 0:
                bbox.width += 1
            if bbox.height % 2 == 0:
                bbox.height += 1
            roi = get_roi(bbox, squared = squared) # get roi slightly bigger that bbox
            tracker.start(image, bbox, roi) # first frame approach
        else:
            tracker.detect(image)
            tracker.update()
        bboxes.append(copy(tracker.bbox))
    ious = dataset.calculate_per_frame_iou(idx, bboxes)
    print("mIou:", np.mean(ious))
    results[idx] = np.mean(ious)

----------------------------Evaluating video named Basketball----------------------------------
mIou: 0.005521620142309797
----------------------------Evaluating video named Biker----------------------------------
mIou: 0.0933969282866821
----------------------------Evaluating video named BlurBody----------------------------------
mIou: 0.04733161822040295
----------------------------Evaluating video named BlurCar3----------------------------------
mIou: 0.011330748020607937
----------------------------Evaluating video named Bolt----------------------------------
mIou: 0.009908925318761384
----------------------------Evaluating video named Box----------------------------------
mIou: 0.009122561358786495
----------------------------Evaluating video named CarScale----------------------------------
mIou: 0.014082314469911369
----------------------------Evaluating video named Coke----------------------------------
mIou: 0.0272316639523316
----------------------------Evaluating video named 

In [10]:
result_file = f"../results/ious/experiments/2/best_variation_layers/mobilenet/mobilenet_squared_last_layer.json"

In [11]:
with open(result_file, 'w') as fp:
    json.dump(results, fp)

# 2) Load results

In [12]:
with open(result_file) as fp:
    data = json.load(fp)

In [13]:
mIoU = 0

In [14]:
for k,v in data.items():
    mIoU += v

In [15]:
k 

'29'

In [16]:
mIoU / 30

0.046444475197535376